In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from IPython.display import display


In [ ]:
### UNCOMMENT FOR PREPROCESSING
# with open("data.csv", "r") as infile, open("processed.csv", "w") as outfile:
#     for line in infile:
#         splited = line.split(',')[3]
#         outfile.write(splited.upper())
#         outfile.write('\n')

In [ ]:
G = nx.Graph()
with open("processed.csv", "r") as infile:
    for k, line in enumerate(infile):
        splited = map(str.strip, line.split('&'))
        for i in xrange(len(splited)):
            G.add_node(splited[i])
            for j in xrange(i+1, len(splited)):
                G.add_edge(splited[i], splited[j])


## A partir del archivo de datos, genere el grafo de colaboraciones entre autores. Cuente el número de nodos y aristas. Haga una representación gráfica del grafo

In [ ]:
plt.subplots(figsize=(20,20))
nx.draw_shell(G)

plt.show()

print 'Número de nodos:', nx.number_of_nodes(G)
print 'Número de aristas:', nx.number_of_edges(G)

## Grafique la distribución del grado de los autores, en escalas lineales, semilogarítmica, y log-log. Argumente, a partir de los gráficos, de qué tipo de distribución se trata

In [ ]:
degree_sequence = sorted([n[1] for n in nx.degree(G)], reverse=True)
plt.hist(degree_sequence, bins=50)
plt.title('Distribution Degree')
plt.xlabel('Degree')
plt.ylabel('# of Nodes')
plt.show()

_, axs = plt.subplots(1, 2, figsize = (20, 5))

h , b = np.histogram(degree_sequence, bins=50)
axs[0].semilogy(h)
axs[0].set_title('Semilog')
axs[1].loglog(h)
axs[1].set_title('Log-Log')
plt.show()


RESPONDER

## Calcule el número de componentes conexas del grafo. Muestre el tamaño de la componente mayor, o componente gigante

In [ ]:
connected_components = nx.connected_component_subgraphs(G)

graphs = list(connected_components)
print 'Número de componentes conexas:', len(graphs)
max_component = max(graphs, key=len)
print 'Tamaño de la componente gigante:', nx.number_of_nodes(max_component)

## Trabajando con la componente gigante del grafo, estudie, parándose en un nodo al azar, cómo aumenta el número de autores alcanzados a medida que se aleja del nodo semilla. Grafique el número de autores alcanzados en función de la distancia al nodo semilla. Grafique también el número de nuevos autores que se agregan en cada paso, y estime el máximo de esta función. Pruebe con varios nodos semillas y analice la robustez de este resultado. Discuta el significado de los gráficos y su relación con el fenómeno de seis grados de separación

In [ ]:
# TODO

## Compute el coeficiente de clustering C y el camino mínimo medio l para la componente gigante. Genere un grafo aleatorio con la misma distribución de grado y compute las mismas medidas para este grafo. Compare e interprete los resultados. ¿Se trata de un grafo con estructura de mundos pequeños?

## Discuta cómo haría para individuar a los autores "estrella" del campo. Evalúe quiénes son estos autores según por lo menos dos métricas diferentes. Analice qué sucedería con la comunidad si estos autores desapareciesen. En particular, determine cuántos autores deberían desaparecer para que desaparezca la componente gigante del grafo